<a href="https://colab.research.google.com/github/JJChrzanowski/GlyCulator3_hotfix/blob/main/TITR_GlyCulator_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## TITR rerun analysis with GlyCulator API

Please follow these steps:

*   Enter your original_session_id and api_key in the variables below.
*   Run this cell.

The code will:

*   Connect to the GlyCulator API
*   Retrieve the original analysis details
*   Perform analysis with TITR for each file and date range
*   Show you the final results as a table
*   Provide a link to download the results as a CSV file



In [ ]:
# @title
import requests
import time
import datetime
import pandas as pd

def get_analysis_details(session_id, api_key, base_url="https://glyculator.btm.umed.pl"):
    url = f"{base_url}/api/analyses/{session_id}?api_key={api_key}"
    response = requests.get(url, headers={"accept": "application/json"})
    response.raise_for_status()
    return response.json()

def create_analysis_with_retries(api_key, data, base_url="https://glyculator.btm.umed.pl", max_retries=3, wait_seconds=30):
    """Create a new analysis with retries on 500 Internal Server Error."""
    url = f"{base_url}/api/analyses/new?api_key={api_key}"
    headers = {"Content-Type": "application/json", "accept": "application/json"}

    for attempt in range(max_retries):
        try:
            response = requests.post(url, json=data, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.HTTPError as e:
            # Check if it's a 500 error
            if response.status_code == 500:
                print(f"Got 500 Internal Server Error (attempt {attempt+1}/{max_retries}). Waiting {wait_seconds} seconds and retrying...")
                time.sleep(wait_seconds)
            else:
                raise e
    raise RuntimeError("Failed to create analysis after multiple retries due to 500 errors.")

def wait_for_analysis_finish(session_id, api_key, base_url="https://glyculator.btm.umed.pl", timeout=1200, interval=15):
    """Wait longer and poll less frequently if analysis takes time."""
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_analysis_details(session_id, api_key, base_url=base_url)
        if details.get("finished"):
            return details
        time.sleep(interval)
    raise TimeoutError(f"Analysis {session_id} did not finish within {timeout} seconds.")

def flatten_indices(indices):
    flat = {}
    for period in ["whole", "day", "night"]:
        if period in indices:
            for k, v in indices[period].items():
                flat[f"{period}_{k}"] = v
    return flat

def compute_titr_indices(original_session_id, api_key):
    # Get the original analysis details
    original_details = get_analysis_details(original_session_id, api_key)

    # Extract configuration parameters
    imputation_method = original_details.get("imputation_method")
    imputation_max_gap = original_details.get("imputation_max_gap")
    imputation_padding = original_details.get("imputation_padding")
    tbr_low = original_details.get("tbr_low")
    tar_high = original_details.get("tar_high")
    glucose_unit = original_details.get("glucose_unit")
    night_start = original_details.get("night_start")
    night_end = original_details.get("night_end")

    original_name = original_details.get("name", "Original_Analysis")
    original_analysis_files = original_details.get("analysis_files", [])

    # Map original data
    original_data_map = {}
    for f in original_analysis_files:
        file_hash = f["file"]["file_hash"]
        filename = f["file"]["filename"]
        date_from = f["date_from"]
        date_to = f["date_to"]
        original_indices = f["indices"]
        original_flat = flatten_indices(original_indices)

        original_data_map[(file_hash, date_from, date_to)] = {
            "filename": filename,
            "file_hash": file_hash,
            "date_from": date_from,
            "date_to": date_to,
            **original_flat
        }

    # Prepare new analysis with modified tbr_high and tar_low for all files at once
    new_analysis_data = {
        "name": f"{original_name}_TightRange",
        "imputation_method": imputation_method,
        "imputation_max_gap": imputation_max_gap,
        "imputation_padding": imputation_padding,
        "tbr_low": tbr_low,
        "tbr_high": 70,
        "tar_low": 140,
        "tar_high": tar_high,
        "glucose_unit": glucose_unit,
        "night_start": night_start,
        "night_end": night_end,
        "analysis_files": []
    }

    # Add all files
    for f in original_analysis_files:
        file_hash = f["file"]["file_hash"]
        date_from = f["date_from"]
        date_to = f["date_to"]
        new_analysis_data["analysis_files"].append({
            "date_from": date_from,
            "date_to": date_to,
            "file": {
                "file_hash": file_hash
            }
        })

    # Create new analysis with retries on 500 error
    creation_response = create_analysis_with_retries(api_key, new_analysis_data)
    new_session_id = creation_response.get("session_id")
    if not new_session_id:
        raise RuntimeError("No session_id returned from new tight-range analysis creation.")

    # Wait for the new analysis to finish (extended timeout and interval)
    finished_details = wait_for_analysis_finish(new_session_id, api_key)

    new_analysis_files = finished_details.get("analysis_files", [])

    records = []
    for new_file in new_analysis_files:
        file_hash = new_file["file"]["file_hash"]
        filename = new_file["file"]["filename"]
        date_from = new_file["date_from"]
        date_to = new_file["date_to"]
        tight_indices = new_file["indices"]
        tight_flat = flatten_indices(tight_indices)

        # Extract tight-range tir metrics and rename
        whole_titr = tight_flat.get("whole_tir")
        day_titr = tight_flat.get("day_tir")
        night_titr = tight_flat.get("night_tir")

        key = (file_hash, date_from, date_to)
        if key not in original_data_map:
            continue

        original_record = original_data_map[key].copy()
        original_record["whole_titr"] = whole_titr
        original_record["day_titr"] = day_titr
        original_record["night_titr"] = night_titr

        records.append(original_record)

    df = pd.DataFrame(records)
    return df

In [ ]:
# NEVER SHARE YOUR API KEY!!!
original_session_id = "YOUR_SESSION_ID"
api_key = "YOUR_API_KEY"

In [ ]:
# Example use
df_results = compute_titr_indices(original_session_id, api_key, cleanup=False)
df_results.head()